In [1]:
import gradio as gr
import tf_keras as keras
from transformers import pipeline
import json



/opt/anaconda3/lib/python3.11/site-packages/starlette/formparsers.py:12: FutureWarning: Please use `import python_multipart` instead.
  import multipart


In [2]:
# Load a pre-trained text generation model from Hugging Face
model = pipeline("text-generation", model="gpt2")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [3]:
# Function to get model response and collect user feedback
def generate_response(user_input, feedback=None):
    # Generate model response
    response = model(user_input, max_length=50, num_return_sequences=1)[0]["generated_text"]

    # Save the feedback to a JSON file for simplicity
    if feedback is not None:
        feedback_data = {
            "user_input": user_input,
            "response": response,
            "feedback": feedback
        }
        with open("feedback.json", "a") as f:
            f.write(json.dumps(feedback_data) + "\n")

    return response


In [4]:
# Gradio interface function to manage feedback and display response
def feedback_interface(user_input, feedback=None):
    response = generate_response(user_input, feedback)
    return response


In [5]:
# Create Gradio UI
with gr.Blocks() as interface:
    user_input = gr.Textbox(label="User Input")
    response_output = gr.Textbox(label="Response")
    
    # Define feedback buttons
    thumbs_up = gr.Button("👍 Thumbs Up")
    thumbs_down = gr.Button("👎 Thumbs Down")

    def on_submit(user_input):
        response = feedback_interface(user_input)
        response_output.value = response

    # Define actions for thumbs up and thumbs down feedback
    thumbs_up.click(lambda: feedback_interface(user_input.value, "Positive"), inputs=[], outputs=response_output)
    thumbs_down.click(lambda: feedback_interface(user_input.value, "Negative"), inputs=[], outputs=response_output)

    # Display layout
    gr.Row([user_input, response_output, thumbs_up, thumbs_down])

interface.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
